<a href="https://colab.research.google.com/github/analyticsworld1/nlp_Recommendation_SystemAbk/blob/main/Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pandas


In [2]:
import pandas as pd

df = pd.read_csv('/content/sample_data/nlp_datafiles/anime.csv')

df = df.dropna()
df['description'] = df['name'] +' '+ df['genre'] + ' ' +df['type']+' episodes: '+ df['episodes']

<ipython-input-2-9462656b9340>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['description'] = df['name'] +' '+ df['genre'] + ' ' +df['type']+' episodes: '+ df['episodes']


In [3]:
# from transformers import AutoTokenizer, AutoModel
# import torch
# import torch.nn.functional as F

# def mean_pooling(model_output, attention_mask):
#     token_embeddings = model_output[0] #First element of model_output contains all token embeddings
#     input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
#     return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

# tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
# model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

# def get_embeddings(sentences):
#   encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

#   with torch.no_grad():
#       model_output = model(**encoded_input)

#   sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

#   sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

#   return sentence_embeddings

In [4]:
# sentences = ['Some great movie', 'Another funny movie']
# result = get_embeddings(sentences)
# print("Sentence embeddings:")
# print(result.dtype)

In [5]:
# print(result)

In [ ]:
!pip install -U sentence-transformers

In [7]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

anime_embeddings = model.encode(df['description'].tolist())

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
from sklearn.metrics.pairwise import cosine_similarity

def get_recommendations(query, embeddings, df, top_n=5):
    query_embedding = model.encode([query])
    similarities = cosine_similarity(query_embedding, embeddings)
    top_indices = similarities[0].argsort()[-top_n:][::-1]
    return df.iloc[top_indices]

In [9]:
query = "horror movie"
recommendations = get_recommendations(query, anime_embeddings, df)


In [10]:
recommendations

,anime_id,name,genre,type,episodes,rating,members,description
7815,33042,Zonmi-chan: Halloween☆Special Movie!,"Comedy, Horror",ONA,1,4.94,276,"Zonmi-chan: Halloween☆Special Movie! Comedy, H..."
7322,8682,Television,Horror,ONA,1,5.55,322,Television Horror ONA episodes: 1
5861,1494,Darkside Blues,"Horror, Mystery, Psychological, Sci-Fi",Movie,1,6.33,5224,"Darkside Blues Horror, Mystery, Psychological,..."
9535,7751,Midnight Horror School,Comedy,TV,52,6.90,328,Midnight Horror School Comedy TV episodes: 52
6298,34048,Seoul-yeok,"Horror, Thriller",Movie,1,6.18,405,"Seoul-yeok Horror, Thriller Movie episodes: 1"


In [ ]:
recommendations[['name', 'genre', 'description']]

,name,genre,description
7363,Sentou Yousei Shoujo Tasukete! Mave-chan,"Comedy, Parody, Sci-Fi, Shounen, Super Power",Sentou Yousei Shoujo Tasukete! Mave-chan Comed...
8140,Anime TV de Hakken! Tamagotchi,"Comedy, Fantasy, Kids, Slice of Life","Anime TV de Hakken! Tamagotchi Comedy, Fantasy..."
4294,SKET Dance: SD Character Flash Anime,"Comedy, School, Shounen","SKET Dance: SD Character Flash Anime Comedy, S..."
1061,Isshuukan Friends.,"Comedy, School, Shounen, Slice of Life","Isshuukan Friends. Comedy, School, Shounen, Sl..."
2850,Oshiete! Galko-chan,"Comedy, School, Slice of Life","Oshiete! Galko-chan Comedy, School, Slice of L..."


In [ ]:
query = "horror movie"
recommendations = get_recommendations(query, anime_embeddings, df)
recommendations

,anime_id,name,genre,type,episodes,rating,members,description
7815,33042,Zonmi-chan: Halloween☆Special Movie!,"Comedy, Horror",ONA,1,4.94,276,"Zonmi-chan: Halloween☆Special Movie! Comedy, H..."
7322,8682,Television,Horror,ONA,1,5.55,322,Television Horror ONA episodes: 1
5861,1494,Darkside Blues,"Horror, Mystery, Psychological, Sci-Fi",Movie,1,6.33,5224,"Darkside Blues Horror, Mystery, Psychological,..."
9535,7751,Midnight Horror School,Comedy,TV,52,6.90,328,Midnight Horror School Comedy TV episodes: 52
6298,34048,Seoul-yeok,"Horror, Thriller",Movie,1,6.18,405,"Seoul-yeok Horror, Thriller Movie episodes: 1"
